In [1]:
import numpy as np
import gzip
import cPickle

In [2]:
import six

In [3]:
DATA_FILENAME = '/Users/merlin/Documents/Neural_Net_Newbies/data/mnist.pkl.gz'

### im2col

In [4]:
f = gzip.open(DATA_FILENAME,'rb')
train_set, valid_set, test_set = cPickle.load(f)
f.close()

In [5]:
train_set_x, train_set_y = train_set

In [89]:
oneEx = train_set_x[0]

In [92]:
img = oneEx.reshape(1,1,28,28)

In [93]:
label = train_set_y[0]

In [94]:
n, c, h, w = img.shape
# num examples, channels in, height, width

In [95]:
ph = 1
pw = 1
sy = 3
sx = 3
pval = False

In [96]:
img = np.pad(img,((0, 0), (0, 0), (ph, ph + sy - 1), (pw, pw + sx - 1)),
mode='constant', constant_values=(pval,))

In [97]:
img.shape

(1, 1, 32, 32)

In [98]:
kh=9
kw=9

In [99]:
out_h = (h-kh+ph*2)/sy + 1

In [100]:
out_h

8

In [101]:
out_w=out_h

In [102]:
col = np.ndarray((n,c,kh,kw,out_h,out_w), dtype=img.dtype)

In [103]:
col.shape

(1, 1, 9, 9, 8, 8)

In [104]:
for i in six.moves.range(kh): 
    i_lim = i + sy * out_h
    for j in six.moves.range(kw):
        j_lim = j + sx * out_w
        get = img[:, :, i:i_lim:sy, j:j_lim:sx]
        col_assign = col[:, :, i, j, :, :]
        col[:, :, i, j, :, :] = get

In [105]:
col_assign.shape

(1, 1, 8, 8)

In [106]:
print i_lim, j_lim

32 32


In [107]:
get.shape

(1, 1, 8, 8)

In [108]:
col.shape

(1, 1, 9, 9, 8, 8)

In [109]:
weights = np.random.rand(20,1,9,9)

In [110]:
weights.shape

(20, 1, 9, 9)

In [111]:
result = np.tensordot(col, weights, ([1, 2, 3], [1, 2, 3]))

In [112]:
result.shape

(1, 8, 8, 20)

In [113]:
col.shape

(1, 1, 9, 9, 8, 8)

In [114]:
bias = np.random.rand(20,)

In [115]:
bias.shape

(20,)

In [116]:
result += bias

In [117]:
result.shape

(1, 8, 8, 20)

In [118]:
featureMapWidth = (w-kw+pw*2)/sx + 1
featureMapHeight = (h-kh+ph*2)/sy + 1 

In [119]:
featureMapHeight

8

### Checking activations & backprop - CNN

In [160]:
np.set_printoptions(threshold=np.nan)

In [248]:
new_val = np.array([[[[1,1,1,1,1,1,1,1],[2,2,2,2,2,2,2,2],[3,3,3,3,3,3,3,3],[4,4,4,4,4,4,4,4],[1,1,1,1,1,1,1,1],[2,2,2,2,2,2,2,2],[3,3,3,3,3,3,3,3],[4,4,4,4,4,4,4,4]]]])

In [276]:
new_val.ravel()

array([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4])

In [224]:
# can't see rollign if use smaller version
#new_val = np.array([[[[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]]])

In [249]:
new_val.shape

(1, 1, 8, 8)

In [250]:
n, c, h, w = new_val.shape
kh=2
kw=2
ph = 0
pw = 0
sy = 2
sx = 2
depth = 2

In [251]:
def im2col(img, kh, kw, sy, sx, ph, pw, pval=0, cover_all=False):
    n, c, h, w = img.shape
    featureMapWidth = (w-kw+pw*2)/sx + 1
    featureMapHeight = (h-kh+ph*2)/sy + 1 

    img = np.pad(img, ((0, 0), (0, 0), (ph, ph + sy - 1), (pw, pw + sx - 1)),
                    mode='constant', constant_values=(pval,))
    col = np.ndarray((n, c, kh, kw, featureMapHeight, featureMapWidth), dtype=img.dtype)

    for i in six.moves.range(kh):
        i_lim = i + sy * featureMapHeight
        for j in six.moves.range(kw):
            j_lim = j + sx * featureMapWidth
            col[:, :, i, j, :, :] = img[:, :, i:i_lim:sy, j:j_lim:sx]
    return col

def col2im(col, sy, sx, ph, pw, h, w):
    n, c, kh, kw, out_h, out_w = col.shape

    img = np.zeros((n, c, h + 2 * ph + sy - 1, w + 2 * pw + sx - 1), dtype=col.dtype)
    for i in six.moves.range(kh):
        i_lim = i + sy * out_h
        for j in six.moves.range(kw):
            j_lim = j + sx * out_w
            img[:, :, i:i_lim:sy, j:j_lim:sx] += col[:, :, i, j, :, :]

    return img[:, :, ph:h + ph, pw:w + pw]

In [252]:
# W = np.random.rand(depth, c, kh, kw)
# b = np.random.rand(depth,)

W = np.array([[[[ 0.5,  0.5 ], [ 0.5,  0.5]]], [[[ 0.5,  0.5], [ 0.5,  0.5]]]])
b = np.array([ 1,  1])

In [274]:
W.shape

(2, 1, 2, 2)

In [254]:
featureMapWidth = (w-kw+pw*2)/sx + 1
featureMapHeight = (h-kh+ph*2)/sy + 1 

In [255]:
padded = np.pad(new_val , ((0, 0), (0, 0), (ph, ph + sy - 1), (pw, pw + sx - 1)),
                    mode='constant', constant_values=(pval,))

In [256]:
padded.shape

(1, 1, 9, 9)

In [257]:
padded

array([[[[1, 1, 1, 1, 1, 1, 1, 1, 0],
         [2, 2, 2, 2, 2, 2, 2, 2, 0],
         [3, 3, 3, 3, 3, 3, 3, 3, 0],
         [4, 4, 4, 4, 4, 4, 4, 4, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0],
         [2, 2, 2, 2, 2, 2, 2, 2, 0],
         [3, 3, 3, 3, 3, 3, 3, 3, 0],
         [4, 4, 4, 4, 4, 4, 4, 4, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0]]]])

In [258]:
col = np.ndarray((n, c, kh, kw, featureMapHeight, featureMapWidth), dtype=img.dtype)

In [259]:
col.shape

(1, 1, 2, 2, 4, 4)

In [260]:
for i in six.moves.range(kh):
    i_lim = i + sy * featureMapHeight
    for j in six.moves.range(kw):
        j_lim = j + sx * featureMapWidth
        col[:, :, i, j, :, :] = img[:, :, i:i_lim:sy, j:j_lim:sx]

In [261]:
col 

array([[[[[[ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.]],

          [[ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.]]],


         [[[ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.]],

          [[ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.]]]]]], dtype=float32)

In [262]:
col = im2col(new_val, kh, kw, sy, sx, ph, pw)

In [263]:
col #convolved

array([[[[[[1, 1, 1, 1],
           [3, 3, 3, 3],
           [1, 1, 1, 1],
           [3, 3, 3, 3]],

          [[1, 1, 1, 1],
           [3, 3, 3, 3],
           [1, 1, 1, 1],
           [3, 3, 3, 3]]],


         [[[2, 2, 2, 2],
           [4, 4, 4, 4],
           [2, 2, 2, 2],
           [4, 4, 4, 4]],

          [[2, 2, 2, 2],
           [4, 4, 4, 4],
           [2, 2, 2, 2],
           [4, 4, 4, 4]]]]]])

In [367]:
col.shape

(1, 1, 2, 2, 4, 4)

In [264]:
y = np.tensordot(col, W, ([1, 2, 3], [1, 2, 3]))

In [338]:
W.shape

(2, 1, 2, 2)

In [302]:
y.shape

(1, 4, 4, 2)

In [266]:
if b is not None: new_y = y+ b

In [283]:
new_y.ravel()

array([ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  8.,  8.,  8.,  8.,  8.,
        8.,  8.,  8.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  8.,  8.,
        8.,  8.,  8.,  8.,  8.,  8.])

In [268]:
new_y.shape

(1, 4, 4, 2)

In [269]:
y2 = np.rollaxis(new_y, 3, 1)

In [284]:
y2.ravel()

array([ 4.,  4.,  4.,  4.,  8.,  8.,  8.,  8.,  4.,  4.,  4.,  4.,  8.,
        8.,  8.,  8.,  4.,  4.,  4.,  4.,  8.,  8.,  8.,  8.,  4.,  4.,
        4.,  4.,  8.,  8.,  8.,  8.])

In [290]:
y2.shape

(1, 2, 4, 4)

In [291]:
y2.shape

(1, 2, 4, 4)

In [292]:
y2

array([[[[ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.],
         [ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.]],

        [[ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.],
         [ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.]]]])

In [331]:
def sigmoid(x):
        return 1 / (1 + np.exp(-x))

In [332]:
sigmoid = np.vectorize(sigmoid)

In [333]:
r = sigmoid(y2)

In [336]:
r.shape

(1, 2, 4, 4)

#### Backprop

In [285]:
def col2im_cpu(col, sy, sx, ph, pw, h, w):
    n, c, kh, kw, out_h, out_w = col.shape

    img = np.zeros((n, c, h + 2 * ph + sy - 1, w + 2 * pw + sx - 1), dtype=col.dtype)
    for i in six.moves.range(kh):
        i_lim = i + sy * out_h
        for j in six.moves.range(kw):
            j_lim = j + sx * out_w
            img[:, :, i:i_lim:sy, j:j_lim:sx] += col[:, :, i, j, :, :]

    return img[:, :, ph:h + ph, pw:w + pw]

def backward_cpu(x, gy):
    if gb is not None:
        gb += gy[0].sum(axis=(0, 2, 3))
    gW += np.tensordot(gy[0], col, ([0, 2, 3], [0, 4, 5]))
    gcol = np.tensordot(W, gy[0], (0, 1))
    gcol = np.rollaxis(gcol, 3)

    h, w = x[0].shape[2:]
    return col2im(gcol, sy, sx, ph, pw, h, w)

def backprop(epsilon):
    if gradientBias is not None:
        gradientBias += delta[0].sum(axis=(0, 2, 3))
    # get z, activationDerivative, * to the epsilon for the delta
    delta = np.rollaxis(delta, 3) ??

    weight_gradient += np.tensordot(delta[0], colIn, ([0, 2, 3], [0, 4, 5]))
    nextEpsilon = np.tensordot(W, delta[0], (0, 1))
#     epsilon = np.rollaxis(epsilon, 3)
    
    
    h, w = inP[0].shape[2:]
    return col2im(?, sy, sx, ph, pw, h, w)

In [301]:
reverse_input = y2

In [368]:
# delta
gy = np.array([[[[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ]], [[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ]]]])
gb = np.array([1,1])

In [386]:
gy.shape

(1, 2, 4, 4)

In [370]:
gy[0].sum(axis = (1,2)) # same as gy.sum(axis = (0,2,3))

array([32, 32])

In [371]:
gb = gy[0].sum(axis = (1,2)) + gb

In [373]:
gb

array([33, 33])

In [374]:
gW = np.tensordot(gy, col, ([0, 2, 3], [0, 4, 5]))

In [385]:
gW

array([[[[64, 64],
         [96, 96]]],


       [[[64, 64],
         [96, 96]]]])

In [376]:
gW.shape

(2, 1, 2, 2)

In [378]:
W.shape

(2, 1, 2, 2)

In [379]:
gcol = np.tensordot(W, gy, (0, 1))

In [381]:
gcol.shape

(1, 2, 2, 1, 4, 4)

In [364]:
gcol.shape

(1, 2, 2, 1, 4, 4)

In [382]:
gcol = np.rollaxis(gcol, 3)

In [383]:
gcol.shape

(1, 1, 2, 2, 4, 4)

##### preOutput

In [390]:
def deriv_sigmoid(x):
    v = np.exp(x) / np.power(1+np.exp(x),2.0)
    v = x * (1-x)
    return v


In [391]:
deriv_sigmoid = np.vectorize(deriv_sigmoid)

In [405]:
z = np.tensordot(col, W, ([1, 2, 3], [1, 2, 3]))

In [406]:
z.shape

(1, 4, 4, 2)

In [408]:
z+=b

In [413]:
z2 = np.rollaxis(z,3,1)

##### delta

In [414]:
deriv_z = deriv_sigmoid(z2)

In [418]:
deriv_z.shape

(1, 2, 4, 4)

In [422]:
epsilon = np.ones((1,2,4,4))

In [425]:
epsilon.shape

(1, 2, 4, 4)

In [474]:
delta2 = np.dot(epsilon, deriv_z)

In [475]:
delta2.shape

(1, 2, 4, 1, 2, 4)

In [495]:
d = np.rollaxis(delta2,4,2)
np.rollaxis(d,4,1).shape

(1, 1, 2, 2, 4, 4)

In [522]:
delta = np.multiply(epsilon, deriv_z)

In [523]:
delta.shape

(1, 2, 4, 4)

In [534]:
delta.ravel()

array([-12., -12., -12., -12., -56., -56., -56., -56., -12., -12., -12.,
       -12., -56., -56., -56., -56., -12., -12., -12., -12., -56., -56.,
       -56., -56., -12., -12., -12., -12., -56., -56., -56., -56.])

In [530]:
delta.sum((0,2,3))

array([-544., -544.])

In [535]:
wG = np.tensordot(delta, col, ([0, 2, 3],[0, 4, 5]))

In [502]:
delta

array([[[[-544., -544.]],

        [[-544., -544.]]]])

In [536]:
wG.shape

(2, 1, 2, 2)

In [538]:
wG.ravel()

array([-1440., -1440., -1984., -1984., -1440., -1440., -1984., -1984.])

In [548]:
delta[0].shape

(2, 4, 4)

In [546]:
W.shape

(2, 1, 2, 2)

In [560]:
# nextEpsilon
r = np.tensordot(W,delta,([0,1]))

In [562]:
r = np.rollaxis(r,3)

In [579]:
r.shape

(1, 1, 2, 2, 4, 4)

In [564]:
r.ravel()

array([-12., -12., -12., -12., -56., -56., -56., -56., -12., -12., -12.,
       -12., -56., -56., -56., -56., -12., -12., -12., -12., -56., -56.,
       -56., -56., -12., -12., -12., -12., -56., -56., -56., -56., -12.,
       -12., -12., -12., -56., -56., -56., -56., -12., -12., -12., -12.,
       -56., -56., -56., -56., -12., -12., -12., -12., -56., -56., -56.,
       -56., -12., -12., -12., -12., -56., -56., -56., -56.])

In [566]:
def col2im(col, sy, sx, ph, pw, h, w):
    n, c, kh, kw, out_h, out_w = col.shape

    img = np.zeros((n, c, h + 2 * ph + sy - 1, w + 2 * pw + sx - 1), dtype=col.dtype)
    for i in six.moves.range(kh):
        i_lim = i + sy * out_h
        for j in six.moves.range(kw):
            j_lim = j + sx * out_w
            img[:, :, i:i_lim:sy, j:j_lim:sx] += col[:, :, i, j, :, :]

    return img[:, :, ph:h + ph, pw:w + pw]

In [575]:
reshapedEp = col2im(r, sy, sx, ph, pw, h, w)

In [576]:
reshapedEp.shape

(1, 1, 8, 8)

In [578]:
reshapedEp.ravel()

array([-12., -12., -12., -12., -12., -12., -12., -12., -12., -12., -12.,
       -12., -12., -12., -12., -12., -56., -56., -56., -56., -56., -56.,
       -56., -56., -56., -56., -56., -56., -56., -56., -56., -56., -12.,
       -12., -12., -12., -12., -12., -12., -12., -12., -12., -12., -12.,
       -12., -12., -12., -12., -56., -56., -56., -56., -56., -56., -56.,
       -56., -56., -56., -56., -56., -56., -56., -56., -56.])

### Checking activations & backprop - Subsampling

##### Activate

In [593]:
def im2col(img, kh, kw, sy, sx, ph, pw, pval=0, cover_all=False):
    n, c, h, w = img.shape
    featureMapWidth = (w-kw+pw*2)/sx + 1
    featureMapHeight = (h-kh+ph*2)/sy + 1 

    img = np.pad(img, ((0, 0), (0, 0), (ph, ph + sy - 1), (pw, pw + sx - 1)),
                    mode='constant', constant_values=(pval,))
    col = np.ndarray((n, c, kh, kw, featureMapHeight, featureMapWidth), dtype=img.dtype)

    for i in six.moves.range(kh):
        i_lim = i + sy * featureMapHeight
        for j in six.moves.range(kw):
            j_lim = j + sx * featureMapWidth
            col[:, :, i, j, :, :] = img[:, :, i:i_lim:sy, j:j_lim:sx]
    return col

In [653]:
def forward_max(x):
    kh=2
    kw=2
    ph = 0
    pw = 0
    sy = 2
    sx = 2
    col = im2col(x, kh, kw, sy, sx, ph, pw)
    n, c, kh, kw, out_h, out_w = col.shape
    col = col.reshape(n, c, kh * kw, out_h, out_w)

    # We select maximum twice, since the implementation using numpy.choose
    # hits its bug when kh * kw >= 32.
    indexes = col.argmax(axis=2)
    y = col.max(axis=2)
    return y

def forward_mean(x):
    kh=2
    kw=2
    ph = 0
    pw = 0
    sy = 2
    sx = 2
    col = im2col(x, kh, kw, sy, sx, ph, pw)
#     n, c, kh, kw, out_h, out_w = col.shape
#     col = col.reshape(n, c, kh * kw, out_h, out_w)

    y = col.mean(axis=2,3)
    return y

In [626]:
y2 = np.array([[[[ 1.,  1.,  3.,  7.],
                 [ 5.,  1.,  3.,  3.],
                 [ 2.,  2.,  8.,  4.],
                 [ 2.,  6.,  4.,  4.]],

                [[ 3.,  3.,  6.,  7.],
                 [ 4.,  4.,  6.,  7.],
                 [ 5.,  5.,  9.,  8.],
                 [ 4.,  4.,  9.,  8.]]]])

In [630]:
y2.shape

(1, 2, 4, 4)

In [654]:
forward_max(y2).shape

(1, 2, 2, 2)

In [624]:
w = y2.shape[2]
kw=2
pw = 0
sx = 2

featureMapWidth = (w-kw+pw*2)/sx + 1

In [625]:
featureMapWidth

2

In [643]:
kh=2
kw=2
ph = 0
pw = 0
sy = 2
sx = 2

col = im2col(y2, kh, kw, sy, sx, ph, pw)

In [644]:
col

array([[[[[[ 1.,  3.],
           [ 2.,  8.]],

          [[ 1.,  7.],
           [ 2.,  4.]]],


         [[[ 5.,  3.],
           [ 2.,  4.]],

          [[ 1.,  3.],
           [ 6.,  4.]]]],



        [[[[ 3.,  6.],
           [ 5.,  9.]],

          [[ 3.,  7.],
           [ 5.,  8.]]],


         [[[ 4.,  6.],
           [ 4.,  9.]],

          [[ 4.,  7.],
           [ 4.,  8.]]]]]])

In [645]:
col.shape

(1, 2, 2, 2, 2, 2)

In [646]:
n, c, kh, kw, out_h, out_w = col.shape

In [647]:
col = col.reshape(n, c, kh * kw, out_h, out_w)


In [648]:
col.shape

(1, 2, 4, 2, 2)

In [651]:
indexes = col.argmax(axis=2)

In [652]:
indexes

array([[[[2, 1],
         [3, 0]],

        [[2, 1],
         [0, 0]]]])

In [649]:
indexes.shape

(1, 2, 2, 2)

In [641]:
y = col.max(axis=2)

In [657]:
y2

array([[[[ 1.,  1.,  3.,  7.],
         [ 5.,  1.,  3.,  3.],
         [ 2.,  2.,  8.,  4.],
         [ 2.,  6.,  4.,  4.]],

        [[ 3.,  3.,  6.,  7.],
         [ 4.,  4.,  6.,  7.],
         [ 5.,  5.,  9.,  8.],
         [ 4.,  4.,  9.,  8.]]]])

In [642]:
y

array([[[[ 5.,  7.],
         [ 6.,  8.]],

        [[ 4.,  7.],
         [ 5.,  9.]]]])

In [758]:
col.mean(axis=(2))

array([[[[ 2. ,  4. ],
         [ 3. ,  5. ]],

        [[ 3.5,  6.5],
         [ 4.5,  8.5]]]])

In [797]:
forward_mean(y2)

array([[[[ 2. ,  4. ],
         [ 3. ,  5. ]],

        [[ 3.5,  6.5],
         [ 4.5,  8.5]]]])

In [661]:
forward_mean(y2).shape

(1, 2, 2, 2)

##### backprop

In [ ]:
def col2im(col, sy, sx, ph, pw, h, w):
    n, c, kh, kw, out_h, out_w = col.shape

    img = np.zeros((n, c, h + 2 * ph + sy - 1, w + 2 * pw + sx - 1), dtype=col.dtype)
    for i in six.moves.range(kh):
        i_lim = i + sy * out_h
        for j in six.moves.range(kw):
            j_lim = j + sx * out_w
            img[:, :, i:i_lim:sy, j:j_lim:sx] += col[:, :, i, j, :, :]

    return img[:, :, ph:h + ph, pw:w + pw]

In [783]:
def backward_max(x, gy):
    n, c, out_h, out_w = gy.shape
    h, w = x.shape[2:]
    kh = 2
    kw = 2
    gcol = np.zeros((n, c, kh, kw, out_h, out_w), dtype=np.float32)

    # TODO(beam2d): Make it fast
    gcol_r = np.rollaxis(gcol.reshape(n, c, -1, out_h, out_w), 2)
    for i in np.ndindex(n, c, out_h, out_w):
        gcol_r[indexes[i]][i] = gy[i]

    gx = col2im(gcol, sy, sx, ph, pw, h, w)
    return gx

def backward_mean(x, gy):
#     n, c, out_h, out_w = gy.shape
    h, w = x.shape[2:]
    kh = 2
    kw = 2

    gcol = np.tile(gy[0][:,:,np.newaxis, np.newaxis], (1, 1, kh, kw, 1, 1))
    gx = col2im(gcol, sy, sx, ph, pw, h, w)
    gx /= (kh * kw)
    return gx

In [695]:
epsilon = np.ones((1,2,2,2))

In [710]:
epsilon.ravel()

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [706]:
y2

array([[[[ 1.,  1.,  3.,  7.],
         [ 5.,  1.,  3.,  3.],
         [ 2.,  2.,  8.,  4.],
         [ 2.,  6.,  4.,  4.]],

        [[ 3.,  3.,  6.,  7.],
         [ 4.,  4.,  6.,  7.],
         [ 5.,  5.,  9.,  8.],
         [ 4.,  4.,  9.,  8.]]]])

In [732]:
backward_max(y2, epsilon).shape

(1, 2, 4, 4)

In [733]:
n, c, out_h, out_w = epsilon.shape
h, w = y2.shape[2:]
kh = 2
kw = 2

gcol = np.zeros((n, c, kh, kw, out_h, out_w), dtype=np.float32)

In [734]:
gcol.shape

(1, 2, 2, 2, 2, 2)

In [735]:
gcol_r = np.rollaxis(gcol.reshape(n, c, -1, out_h, out_w), 2)

In [736]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [701]:
n, c, out_h, out_w = epsilon.shape
for i in np.ndindex(n, c, out_h, out_w):
    gcol_r[indexes[i]][i] = gy[i]

In [717]:
gcol_r[indexes[0,1,1,1]]

array([[[[ 0.,  0.],
         [ 0.,  2.]],

        [[ 0.,  0.],
         [ 2.,  2.]]]], dtype=float32)

In [721]:
epsilon2=np.array([[[[ 1.,  2.],
                     [ 3.,  4.]],

                    [[ 5.,  6.],
                     [ 7.,  8.]]]])

In [719]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [755]:
out_w

2

In [750]:
for i in np.ndindex(n, c, out_h, out_w):
    print gcol_r[i]
    gcol_r[i] = epsilon2[i]
    print gcol_r[i]
    print epsilon2[i]
    print i

[ 1.  1.]
[ 1.  1.]
1.0
(0, 0, 0, 0)
[ 1.  1.]
[ 2.  2.]
2.0
(0, 0, 0, 1)
[ 2.  2.]
[ 3.  3.]
3.0
(0, 0, 1, 0)
[ 3.  3.]
[ 4.  4.]
4.0
(0, 0, 1, 1)


IndexError: index 1 is out of bounds for axis 1 with size 1

In [751]:
gcol_r

array([[[[[ 4.,  4.],
          [ 4.,  4.]],

         [[ 4.,  4.],
          [ 4.,  4.]]]],



       [[[[ 4.,  4.],
          [ 4.,  4.]],

         [[ 4.,  4.],
          [ 4.,  4.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]]], dtype=float32)

In [749]:
gcol_r[i]

array([ 1.,  1.], dtype=float32)

In [739]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [729]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [743]:
gcol_r

array([[[[[ 1.,  1.],
          [ 2.,  2.]],

         [[ 3.,  3.],
          [ 4.,  4.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]]], dtype=float32)

In [731]:
indexes.shape

(1, 2, 2, 2)

In [763]:
epsilon2.shape

(1, 2, 2, 2)

In [765]:
epsilon2[0].shape

(2, 2, 2)

In [767]:
epsilon2[:,:,np.newaxis, np.newaxis].shape

(1, 2, 1, 1, 2, 2)

In [768]:
epsilon2

array([[[[ 1.,  2.],
         [ 3.,  4.]],

        [[ 5.,  6.],
         [ 7.,  8.]]]])

In [773]:
np.tile(epsilon2[0][:,:,np.newaxis, np.newaxis], (1,1,kh,kw,1,1))

array([[[[[[ 1.,  2.]],

          [[ 1.,  2.]]],


         [[[ 3.,  4.]],

          [[ 3.,  4.]]],


         [[[ 1.,  2.]],

          [[ 1.,  2.]]],


         [[[ 3.,  4.]],

          [[ 3.,  4.]]]],



        [[[[ 5.,  6.]],

          [[ 5.,  6.]]],


         [[[ 7.,  8.]],

          [[ 7.,  8.]]],


         [[[ 5.,  6.]],

          [[ 5.,  6.]]],


         [[[ 7.,  8.]],

          [[ 7.,  8.]]]]]])

In [775]:
np.tile(epsilon2[0][:,:,np.newaxis, np.newaxis], (1,1,kh,kw,1,1)).shape

(1, 2, 4, 2, 1, 2)

In [771]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [787]:
# missing col2im for shaping
((np.tile(epsilon2[0][:,:,np.newaxis, np.newaxis], (1,1,kh,kw,1,1)))/(kh*kw)).shape

(1, 2, 4, 2, 1, 2)

In [778]:
epsilon2[0][:,:,np.newaxis, np.newaxis].shape

(2, 2, 1, 1, 2)

In [779]:
epsilon2[:,:,np.newaxis, np.newaxis].shape

(1, 2, 1, 1, 2, 2)

In [780]:
epsilon2.shape

(1, 2, 2, 2)

In [790]:
backward_mean(y2,epsilon2).shape

(1, 2, 4, 4)

In [789]:
epsilon2.ravel()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.])

In [792]:
W.shape

(2, 1, 2, 2)

In [795]:
backward_mean(y2,W).shape

(1, 1, 4, 4)

In [794]:
y2.shape

(1, 2, 4, 4)

In [796]:
backward_max(y2,W).shape

IndexError: index 1 is out of bounds for axis 0 with size 1